In [1]:
#import necessary libraries
from flask import Flask
from flask_restful import Resource, Api, reqparse
from google.cloud import bigquery
import pandas_gbq
import pandas
import json
import os
from geojson import Feature, FeatureCollection, Point

In [3]:
#google credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Hurricane -32cb24436d46.json"

# #create Flass app instance
# app = Flask(__name__)
# api = Api(app)


#setup connection to BigAssQuery
client = bigquery.Client()


# @app.route("/")
# def index():

query = """
    SELECT year, latitude, longitude, sea_surface_temp
    FROM `bigquery-public-data.noaa_icoads.icoads_core_*` 
    WHERE year >=1851 
    AND latitude <=50
    AND latitude >=0
    AND longitude <=-50
    AND longitude >=-100
    ORDER BY year
    LIMIT 5000
"""

query_res = client.query(query) 

In [4]:
import pandas as pd
import pandas_gbq
df = pandas_gbq.read_gbq(query)

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 11315.66rows/s]


In [5]:
lat_bins=range(0, 51, 10)
df['latitudeSquare'] = pd.cut(df['latitude'], bins=lat_bins)

In [6]:
long_bins=range(-100, -49, 10)
df['longitudeSquare']=pd.cut(df['longitude'], bins=long_bins)

In [7]:
dfSquare=df.groupby(['year','latitudeSquare','longitudeSquare']).agg({'sea_surface_temp': ['min','max','mean','median']})
dfSquare.head(100)

sea_surface_temp                        
                                                 min   max       mean median
year latitudeSquare longitudeSquare                                         
1851 (0, 10]        (-100, -90]                 19.4  26.7  23.632353   23.3
                    (-90, -80]                  21.7  27.8  25.133333   24.4
                    (-80, -70]                  25.0  28.9  27.210000   27.5
                    (-70, -60]                   NaN   NaN        NaN    NaN
                    (-60, -50]                  26.1  28.3  26.800000   26.4
     (10, 20]       (-100, -90]                 26.1  28.3  27.511111   27.8
                    (-90, -80]                  27.2  29.4  27.994444   27.8
                    (-80, -70]                  26.7  27.8  27.433333   27.8
                    (-70, -60]                  25.0  27.8  26.227273   26.1
                    (-60, -50]                  23.3  29.7  26.237410   26.1
     (20, 30]       (-100, -90]                 21.7  25.0  23.866667   24.4
                    (-90, -80]                  16.7  29.4  25.708387   26.1
                    (-80, -70]                  16.7  28.3  24.509167   25.0
                    (-70, -60]                  18.9  29.4  25.157399   25.6
                    (-60, -50]                  19.4  30.0  26.123729   26.7
     (30, 40]       (-100, -90]                  NaN   NaN        NaN    NaN
                    (-90, -80]                   NaN   NaN        NaN    NaN
                    (-80, -70]                   5.6  29.4  19.686476   20.6
                    (-70, -60]                   5.0  29.4  21.117863   21.1
                    (-60, -50]                  10.0  29.4  21.292059   21.1
     (40, 50]       (-100, -90]                  NaN   NaN        NaN    NaN
                    (-90, -80]                   NaN   NaN        NaN    NaN
                    (-80, -70]                   2.8  22.2  13.180000   13.3
                    (-70, -60]                  -1.1  26.7  13.136716   13.9
                    (-60, -50]                  -2.2  25.6  12.983002   13.9

In [31]:
dfProcessed = dfSquare.reset_index()
dfProcessed.head()

year latitudeSquare longitudeSquare sea_surface_temp                   \
                                                    min   max       mean   
0  1851        (0, 10]     (-100, -90]             19.4  26.7  23.632353   
1  1851        (0, 10]      (-90, -80]             21.7  27.8  25.133333   
2  1851        (0, 10]      (-80, -70]             25.0  28.9  27.210000   
3  1851        (0, 10]      (-70, -60]              NaN   NaN        NaN   
4  1851        (0, 10]      (-60, -50]             26.1  28.3  26.800000   

          
  median  
0   23.3  
1   24.4  
2   27.5  
3    NaN  
4   26.4

In [36]:
dfProcessed['longitudeSquare'].astype(str).str.replace(r'(','')
dfProcessed['longitudeSquare'].astype(str).str.replace(r']','')
dfProcessed['longitudeSquare'].astype(str).str.replace(r' ','')


0     (-100,-90]
1      (-90,-80]
2      (-80,-70]
3      (-70,-60]
4      (-60,-50]
5     (-100,-90]
6      (-90,-80]
7      (-80,-70]
8      (-70,-60]
9      (-60,-50]
10    (-100,-90]
11     (-90,-80]
12     (-80,-70]
13     (-70,-60]
14     (-60,-50]
15    (-100,-90]
16     (-90,-80]
17     (-80,-70]
18     (-70,-60]
19     (-60,-50]
20    (-100,-90]
21     (-90,-80]
22     (-80,-70]
23     (-70,-60]
24     (-60,-50]
Name: longitudeSquare, dtype: object

In [37]:
# dfProcessed['longitudeSquare']=dfProcessed['longitudeSquare'].str.replace(r'\D','').astype(str)
dfProcessed['longitudeLeft'], dfProcessed['longitudeRight'] = dfProcessed['longitudeSquare'].astype(str).str.split(',')


ValueError: too many values to unpack (expected 2)

In [111]:
split = str(dfProcessed['latitudeSquare'])
splitLat = split
splitLats=split.split(",")
splitLats


['0      (0',
 ' 10]\n1      (0',
 ' 10]\n2      (0',
 ' 10]\n3      (0',
 ' 10]\n4      (0',
 ' 10]\n5     (10',
 ' 20]\n6     (10',
 ' 20]\n7     (10',
 ' 20]\n8     (10',
 ' 20]\n9     (10',
 ' 20]\n10    (20',
 ' 30]\n11    (20',
 ' 30]\n12    (20',
 ' 30]\n13    (20',
 ' 30]\n14    (20',
 ' 30]\n15    (30',
 ' 40]\n16    (30',
 ' 40]\n17    (30',
 ' 40]\n18    (30',
 ' 40]\n19    (30',
 ' 40]\n20    (40',
 ' 50]\n21    (40',
 ' 50]\n22    (40',
 ' 50]\n23    (40',
 ' 50]\n24    (40',
 ' 50]\nName: latitudeSquare',
 ' dtype: category\nCategories (5',
 ' interval[int64]): [(0',
 ' 10] < (10',
 ' 20] < (20',
 ' 30] < (30',
 ' 40] < (40',
 ' 50]]']

In [69]:
smallSstJson= dfProcessed.to_json(path_or_buf="smallSst.json",orient="split", index="false")


In [90]:
# columns used for constructing geojson object
features = dfProcessed.apply(
    lambda row: Feature(geometry="P"(((row['longitudeSquare']), (row['latitudeSquare'])))), axis=1).tolist()

# all the other columns used as properties
properties = df.drop(['latitudeSqure', 'longitudeSquare'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

TypeError: 'str' object is not callable

In [104]:
def df_to_geojson(dfProcessed, properties, lat=str('latitudeSquare'), lon=str('longitudeSquare')):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Polygon',
                               'coordinates':[(])]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson


In [105]:
cols = ['year', ('sea_surface_temp'[0]), ('sea_surface_temp'[1]), ('sea_surface_temp'[2]), ('sea_surface_temp'[3])]
geojson = df_to_geojson(dfProcessed, cols)

KeyError: 's'